# Ngonye Falls Synthetic Historic Flow Series

## Source Data

1. Daily Flow for Victoria Falls 1924/25 - 2016/17 from *Zambezi River Authority*
2. Daily Stage (Level) for Ngonye Falls 2005/06 – 2016/17 from *Zambezi River Authority*
3. Stage - Discharge for Ngonye Falls prepared by Mott MacDonald in a separate study  

## Procedure

1. **Calculate Flow Duration Curves (FDCs) from daily data for the following:**
    1. The target site (Ngonye) for 2005/06 – 2016/17; 
    2. The analogue site (Victoria Falls) for the exact period of data at the target site (2005/06 – 2016/17);
    3. Analogue site for long-term record (1924/25 -2016/17); 
    
    
2. **Compare the FDCs for the analogue site for the two periods and derive factors between the shorter and longer periods;** 

3. **Apply these factors to the target site FDC to produce an estimated long-term FDC for the target site;**

4. **For each daily flow value at the analogue site, determine its position on the long-term FDC (percentile); and,**

5. **Look up the flow value for this percentile from the estimated long-term FDC for the target site taking into account of the 11-day time lag.**


In [1]:
import numpy as np
import pandas as pd

Load the Vic Falls data and add some helper columns.

In [2]:
vicfalls=pd.read_csv("daily_gauge_vicfalls.csv")

vicfalls['Date']=pd.to_datetime(vicfalls['Date'],format="%d/%m/%Y")
vicfalls=vicfalls.set_index(pd.DatetimeIndex(vicfalls['Date']))
vicfalls=vicfalls.drop(['Date'],axis=1)

vicfalls=vicfalls.astype({'Flow': 'float64'})

vicfalls

,Flow
Date,
1924-10-01,100.0
1924-10-02,100.0
1924-10-03,100.0
1924-10-04,100.0
1924-10-05,100.0
...,...
2017-10-27,180.0
2017-10-28,174.0
2017-10-29,168.0


Load the Ngonye Falls level data and calculate flow based on the stage-discharge relationship:

\begin{equation*}
flow=1093.0355*(level-2.85)^{1.659}
\end{equation*}

Add some helper columns.

In [3]:
ngo=pd.read_csv("daily_gauge_ngonye.csv")
ngo['Date']=pd.to_datetime(ngo['Date'],format="%d/%m/%Y")
ngo=ngo.set_index(pd.DatetimeIndex(ngo['Date']))
ngo=ngo.drop(['Date'],axis=1)
ngo['Flow']=1093.0355*(ngo['Level']-2.85)**1.659
ngo['VicFalls']=vicfalls['Flow']
ngo

,Level,Flow,VicFalls
Date,,,
2005-10-01,3.218,208.149704,212.0
2005-10-02,3.216,206.276326,211.0
2005-10-03,3.215,205.342162,209.0
2005-10-04,3.213,203.478889,204.0
2005-10-05,3.212,202.549785,204.0
...,...,...,...
2018-06-01,4.659,2922.300348,NaN
2018-06-02,4.628,2839.690782,NaN
2018-06-03,4.598,2760.644580,NaN


Build the Flow Duration Curve table in 0.1% exceedance increments.

Add FDC flows for the full Vic Falls timeseries.

In [4]:
fdc=pd.DataFrame({'Exceedance': np.arange(0,1.001,0.001)}).set_index('Exceedance')
fdc['VicFalls_full']=np.percentile(vicfalls['Flow'],((1-fdc.index)*100))
fdc

,VicFalls_full
Exceedance,
0.000,9436.000
0.001,8466.000
0.002,7541.252
0.003,6742.240
0.004,6288.312
...,...
0.996,128.000
0.997,115.000
0.998,107.000


Add flows for Ngonye to the FDC.

Add flows for the portion of the Vic Falls series that overlaps with the Ngonye Series.

In [5]:
fdc['Ngonye_gauged']=np.percentile((ngo.dropna())['Flow'],((1-fdc.index)*100))
fdc['VicFalls_overlap']=np.percentile((ngo.dropna())['VicFalls'],((1-fdc.index)*100))

Calculate the ratio of Vic Falls flows for the whole series and the overallping protion across the FDC.

Smooth that ratio (0.7% moving average) except at the tails of the FDC.

In [6]:
fdc['VicFalls_factor']=fdc['VicFalls_full']/fdc['VicFalls_overlap']
fdc['VicFalls_factor_smooth']=fdc['VicFalls_factor'].rolling(7,center=True).mean()
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name<0.01 else x['VicFalls_factor_smooth'])),axis=1)
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name>0.99 else x['VicFalls_factor_smooth'])),axis=1)

Produce a scaled FDC for Ngonye by using the factors calculated for the Vic Falls data.

In [7]:
fdc['Ngonye_scaled']=fdc['Ngonye_gauged']*fdc['VicFalls_factor_smooth']

Calculate a conversion factor for each row of the FDC between the scaled Ngonye FDC flows and the full Vic Falls series FDC.

In [8]:
fdc['Conversion']=fdc['Ngonye_scaled']/fdc['VicFalls_full']
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion
Exceedance,,,,,,,
0.000,9436.000,5935.920844,5651.000,1.669793,1.669793,9911.758819,1.050420
0.001,8466.000,5829.536151,5590.000,1.514490,1.514490,8828.775144,1.042851
0.002,7541.252,5723.913552,5496.960,1.371895,1.371895,7852.608446,1.041287
0.003,6742.240,5571.640116,5328.220,1.265383,1.265383,7050.259722,1.045685
0.004,6288.312,5459.905479,5261.612,1.195130,1.195130,6525.298548,1.037687
...,...,...,...,...,...,...,...
0.996,128.000,167.349759,154.000,0.831169,0.831169,139.095903,1.086687
0.997,115.000,165.930460,152.000,0.756579,0.756579,125.539492,1.091648
0.998,107.000,164.218852,147.000,0.727891,0.727891,119.533450,1.117135


Lookup the conversion factors from the FDC based on flow and give each record in the full Vic Falls series its corresponding conversion factor. 

In [9]:
if 'Conversion' in vicfalls.columns:
    display(vicfalls.columns)
    vicfalls=vicfalls.drop(['Conversion'],axis=1)
    
tmp=pd.merge_asof(vicfalls.sort_values('Flow').reset_index(),fdc.sort_values('VicFalls_full').reset_index(),left_on='Flow',right_on='VicFalls_full').set_index('Date')
vicfalls['Conversion']=tmp['Conversion']
vicfalls['Exceedance']=tmp['Exceedance']



Prepare the full  synthetic series for Ngonye by applying the 11 day lag to the Vic Falls series and the conversion factors calculated previoulsy.

In [10]:

ngonye_synth=pd.DataFrame(index=vicfalls.index)

ngonye_synth['LaggedDate']=ngonye_synth.index+pd.DateOffset(days=11)
ngonye_synth['VicFalls']=ngonye_synth.join(vicfalls,on='LaggedDate')['Flow']
ngonye_synth['Conversion']=ngonye_synth.join(vicfalls,on='LaggedDate')['Conversion']
ngonye_synth['Flow']=ngonye_synth['Conversion'] * ngonye_synth['VicFalls']

ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124
1924-10-02,1924-10-13,100.0,1.111331,111.133124
1924-10-03,1924-10-14,100.0,1.111331,111.133124
1924-10-04,1924-10-15,100.0,1.111331,111.133124
1924-10-05,1924-10-16,100.0,1.111331,111.133124
...,...,...,...,...
2017-10-27,2017-11-07,NaN,NaN,NaN
2017-10-28,2017-11-08,NaN,NaN,NaN
2017-10-29,2017-11-09,NaN,NaN,NaN


Remove extra columns and delete from the bottom to align to the water year.

In [11]:
ngonye_synth=ngonye_synth.dropna()
ngonye_synth=ngonye_synth.drop(ngonye_synth.loc['2017-10-01':].index)
ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0,1.111331,111.133124
1924-10-02,1924-10-13,100.0,1.111331,111.133124
1924-10-03,1924-10-14,100.0,1.111331,111.133124
1924-10-04,1924-10-15,100.0,1.111331,111.133124
1924-10-05,1924-10-16,100.0,1.111331,111.133124
...,...,...,...,...
2017-09-26,2017-10-07,204.0,0.995184,203.017496
2017-09-27,2017-10-08,204.0,0.995184,203.017496
2017-09-28,2017-10-09,204.0,0.995184,203.017496


Add the new synthetic Ngonye flow series to the FDC.

In [12]:
fdc['Ngonye_synthetic']=np.percentile(ngonye_synth['Flow'],((1-fdc.index)*100))

tmp=pd.merge_asof(ngonye_synth.sort_values('Flow').reset_index(),fdc.sort_values('Ngonye_synthetic').reset_index(),left_on='Flow',right_on='Ngonye_synthetic').set_index('Date')
ngonye_synth['Exceedance']=tmp['Exceedance']

Prepare a subset of the FDC for export

In [13]:
fdc_out=fdc.loc[:,['VicFalls_full','Ngonye_gauged','Ngonye_synthetic','Conversion']]
fdc_out

,VicFalls_full,Ngonye_gauged,Ngonye_synthetic,Conversion
Exceedance,,,,
0.000,9436.000,5935.920844,9911.758819,1.050420
0.001,8466.000,5829.536151,8828.775144,1.042851
0.002,7541.252,5723.913552,7887.651906,1.041287
0.003,6742.240,5571.640116,7005.564014,1.045685
0.004,6288.312,5459.905479,6481.964191,1.037687
...,...,...,...,...
0.996,128.000,167.349759,143.155765,1.086687
0.997,115.000,165.930460,129.906084,1.091648
0.998,107.000,164.218852,124.001990,1.117135


Export to csv

In [14]:
ngonye_synth.to_csv('prepared_ngonye_synthetic.csv')
fdc_out.to_csv('prepared_fdcs.csv')